In [141]:
from dse.cluster import Cluster, GraphExecutionProfile, EXEC_PROFILE_GRAPH_DEFAULT
from dse.graph import GraphOptions, GraphProtocol

graph_name = 'trees'
ep = GraphExecutionProfile(graph_options=GraphOptions(graph_name=graph_name,
                                                  graph_protocol=GraphProtocol.GRAPHSON_3_0))

cluster = Cluster(execution_profiles={EXEC_PROFILE_GRAPH_DEFAULT: ep})
session = cluster.connect()


In [142]:
from __future__ import absolute_import

from dse.util import Point
from dse_graph import DseGraph

from connection import session
from external_data import TOWERS, SENSORS

In [143]:
## notebook specific
from pprint import pprint
from dse.cluster import Cluster, EXEC_PROFILE_GRAPH_SYSTEM_DEFAULT


In [57]:
g = DseGraph.traversal_source(session=session)

In [48]:
result = g.V().hasLabel("Tower").count().next()
print(result)

80


In [85]:
session = cluster.connect()
g = DseGraph.traversal_source(session=session)
for t in range(1):
    for i, (key, value) in enumerate(SENSORS.items()):
        longitude = value[0]
        latitude = value[1]
        distance = 500
        #network = g.V().has('Sensor', 'coordinates', Geo.inside(Point(longitude, latitude), distance, Geo.Unit.METERS)). \
        #                values("sensor_id"). \
        #                toList()
        geo_query = """g.V().has('Sensor', 'coordinates', Geo.inside(Geo.point({}, {}), {}, Geo.Unit.METERS)).
                         values("sensorId").toList();""".format(longitude, latitude, distance)
        network = session.execute_graph(geo_query)
        print(network)
        for r in network:
            print(r)
        break
session.shutdown()
    

In [81]:
for r in network:
    print(r)

In [86]:
geo_query

'g.V().has(\'Sensor\', \'coordinates\', Geo.inside(Geo.point(-117.6276405, 33.5548147), 500, Geo.Unit.METERS)).\n                         values("sensor_id").toList();'

In [101]:
session = cluster.connect()
g = DseGraph.traversal_source(session=session)
geo_query = """g.V().has('Sensor', 'coordinates', Geo.inside(Geo.point({}, {}), {}, Geo.Unit.METERS)).
                         values("sensorId").toList();""".format(longitude, latitude, distance)
        
network = session.execute_graph(geo_query)
print(network)
for r in network:
    print(r)


{'@type': 'g:Int32', '@value': 0}
{'@type': 'g:Int32', '@value': 6}
{'@type': 'g:Int32', '@value': 9}
{'@type': 'g:Int32', '@value': 20}
{'@type': 'g:Int32', '@value': 31}


In [24]:
network.next

<bound method ResultSet.next of <dse.cluster.ResultSet object at 0x108906f28>>

In [100]:
geo_query

'g.V().has(\'Sensor\', \'coordinates\', Geo.inside(Geo.point(-117.6276405, 33.5548147), 500, Geo.Unit.METERS)).\n                         values("sensor_id").toList();'

In [127]:
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [144]:
from external_data import TOWERS, SENSORS
tower_neighbors = {}
for i, (key, val) in enumerate(TOWERS.items()):
    distances = {}
    for j, (key2, val2) in enumerate(SENSORS.items()):
        distance = haversine_np(val[0], val[1], val2[0], val2[1])
        distances[key2] = distance
    sorted_distances = sorted(distances.items(), key=lambda kv: kv[1])
    tower_neighbors[key] = sorted_distances[0:5]

In [145]:
tower_neighbors

{'Seattle': [('1035507', 0.04955522315048425),
  ('1306897', 0.14020432509124567),
  ('130336727', 0.17056936205195505),
  ('1303367', 0.17056936205195505),
  ('1299972', 0.2019548614305855)],
 'QueenAnne': [('1243859', 0.4286702809803447),
  ('1031620', 0.6619167746276604),
  ('1011408', 0.7026854291005248),
  ('1032456', 0.8509381320415765),
  ('1248589', 0.8847041664429616)],
 'Magnolia': [('1033100', 2.144494593802575),
  ('1033099', 2.144494593802575),
  ('1243859', 2.642435120173991),
  ('1226708', 3.107799379983919),
  ('1059929', 3.198532926222152)],
 'Fremont': [('1246797', 0.39377141982606223),
  ('1302824', 0.4892800195695204),
  ('1287761', 1.187988104310032),
  ('1300877', 1.2842324918409267),
  ('1280723', 1.4806419435643356)],
 'UofWA': [('1308366', 0.7214658768825877),
  ('1230452', 0.7517129204653918),
  ('1308463', 0.7917451290793862),
  ('1287413', 0.9425325933777648),
  ('1297262', 0.9694736323854705)],
 'PhinneyRidge': [('1297264', 0.1090231662274728),
  ('1280723'

In [146]:
sensor_neighbors = {}
for i, (key, val) in enumerate(SENSORS.items()):
    distances = {}
    for j, (key2, val2) in enumerate(SENSORS.items()):
        if key != key2:
            distance = haversine_np(val[0], val[1], val2[0], val2[1])
            distances[key2] = distance
    sorted_distances = sorted(distances.items(), key=lambda kv: kv[1])
    sensor_neighbors[key] = sorted_distances[0:5]

In [147]:
for key, val in sensor_neighbors.items():
    test = []
    for item in val:
        test.append(str(key)==str(item[0]))
    if any(test):
        print("theres a true in there")
    if all(test):
        print("they are all true")


In [148]:
sensor_tower_neighbors = {}
for i, (key, val) in enumerate(SENSORS.items()):
    distances = {}
    for j, (key2, val2) in enumerate(TOWERS.items()):
        distance = haversine_np(val[0], val[1], val2[0], val2[1])
        distances[key2] = distance
    sorted_distances = sorted(distances.items(), key=lambda kv: kv[1])
    sensor_tower_neighbors[key] = sorted_distances[0:5]

In [149]:
sensor_tower_neighbors

{'121001920': [('Eastgate', 2.042541774824004),
  ('Newcastle', 4.2241960778444385),
  ('MercerIsland', 4.3289381478133775),
  ('Bellevue', 4.622181161208474),
  ('Medina', 6.591552298921109)],
 '128216313': [('Newcastle', 3.1297407385267744),
  ('MercerIsland', 3.4946816974233665),
  ('Eastgate', 4.210415951102134),
  ('SewardPark', 4.654761303915644),
  ('Kennydale', 4.913787568670965)],
 '12100197': [('Newcastle', 2.242432332905524),
  ('Eastgate', 3.996162387467608),
  ('MercerIsland', 4.382351373444981),
  ('Kennydale', 4.417108020813798),
  ('Glencoe', 4.5256800399305765)],
 '104115939': [('Bellevue', 0.8433703383986887),
  ('Medina', 2.204540210107415),
  ('ClydeHill', 2.743358276332401),
  ('CherryCrest', 3.8757149033207816),
  ('MercerIsland', 4.072227318423029)],
 '122974132': [('Eastgate', 2.042541774824004),
  ('Newcastle', 4.2241960778444385),
  ('MercerIsland', 4.3289381478133775),
  ('Bellevue', 4.622181161208474),
  ('Medina', 6.591552298921109)],
 '130336727': [('Seatt

In [99]:
# sensor_tower_neighbors
# sensor_neighbors
for key, val in sensor_tower_neighbors.items():
    for item in val:
        if item[1] > 7:
            print(item[1])
            print (key, item)


7.911504702793
126947610 ('Glencoe', 7.911504702793)
7.81093118573553
10430211 ('Glencoe', 7.81093118573553)


In [134]:
for key, val in sensor_neighbors.items():
    for item in val:
        if item[1] > 3:
            print(item[1])
            print (key, item)

3.2471416262788897
104115939 ('126951211', 3.2471416262788897)
3.8102837943778716
130336714 ('12059326', 3.8102837943778716)
3.2471416262788897
103318117 ('126951211', 3.2471416262788897)
3.2209850234576964
127620712 ('10330844', 3.2209850234576964)
3.6172474931339007
127620712 ('104115939', 3.6172474931339007)
3.6172474931339007
127620712 ('103318117', 3.6172474931339007)
3.6172474931339007
127620712 ('103318129', 3.6172474931339007)
3.4901885403133224
126947610 ('12059326', 3.4901885403133224)
4.785613682228352
126947610 ('121001920', 4.785613682228352)
3.0813669251261984
10314512 ('121001920', 3.0813669251261984)
3.085159399287672
10314533 ('121001920', 3.085159399287672)
3.2471416262788897
126951211 ('104115939', 3.2471416262788897)
3.2471416262788897
126951211 ('103318117', 3.2471416262788897)
3.2471416262788897
126951211 ('103318129', 3.2471416262788897)
3.2471416262788897
103318129 ('126951211', 3.2471416262788897)
3.831471665619452
10430211 ('12059326', 3.831471665619452)
5.157

In [21]:
import random

In [65]:
tower_names = TOWERS.keys()

In [68]:
if "HarborIsland" in tower_names:
    print("YES")

YES


In [44]:
sensor_edges = {}
sensor_time_check = {}
time_dist = {}
for i, (STARTING_SENSOR, val) in enumerate(SENSORS.items()):
    end = random.randint(2, 9)
    # collect stats on the number of edges for this sensor
    if end in time_dist.keys():
        time_dist[end] += 1
    else:
        time_dist[end] = 1
    # will be used to break the cycle to a tower if needed
    # and no self pings
    visited = [STARTING_SENSOR] 
    current_sensor = STARTING_SENSOR
    cycle_detector = 0
    sensor_edges[STARTING_SENSOR] = {}
    tower_path = False
    for t in range(0, end):
        # First, we need to see if there already is an edge coming out from the current_sensor at time t
        # because if there is an edge, 
        # this means there already is a path from the current_sensor to a tower
        if(current_sensor in sensor_time_check.keys()):
            if(t in sensor_time_check[current_sensor]):
                # this means that we already have a path from this sensor @ this time to a tower
                tower_path = True
                break;
        # If we get here, this means that we need to find another sensor to ping at time t
        # we want to make sure that the next sensor doesn't make a cycle, 
        # so we keep track of them along the way
        cycle_detector = 0
        while(cycle_detector < 5): ## 5 because we have 5 close neighbors in each sensor neighborhood
            # find a neighbor from current sensor using sensor_neighbors
            options = sensor_neighbors[current_sensor]
            # randomly pick a close neighbor
            random.shuffle(options)
            to_edge = options[0][0]
            if to_edge in visited:
                # this means we have included this sensor in this path so far
                # we don't want to add this edge because it will make cycles
                # augment cycle detector to keep this from an infinite loop
                cycle_detector += 1
            else: 
                # this means we have found a new sensor that we haven't visited far
                visited.append(to_edge)
                # we want to keep track of the path we are building up
                # this path begins at STARTING_SENSOR
                sensor_edges[STARTING_SENSOR][t] = to_edge
                # next, we want to track that there is an edge at time t
                # that LEAVES current_sensor to go somewhere.
                # we don't care where the edge goes because what matters is that we stored an edge
                # at time t that LEAVES current_sensor. 
                if current_sensor in sensor_time_check.keys():
                    sensor_time_check[current_sensor].append(t)
                else:
                    sensor_time_check[current_sensor] = [t]
                # we break the while loop for finding a new neighbor
                # print("starting_sensor: ", STARTING_SENSOR)
                # print("current_sensor: ", current_sensor)
                # print("sensor_time_check: ", sensor_time_check)
                # print("sensor_edges: ", sensor_edges)
                # input("Press Enter to continue...")
                current_sensor = to_edge
                break
        # if cycle_detector < end, then we sucessfully found an edge
        # This means that we just want to continue with a new t value
        if(cycle_detector >= 5):
            # else, we never found another edge because we have a cycle
            # this means that we want to break from the t loop and just connect to a tower
            break 
    # from the last sensor on the path, ping a tower
    if not(tower_path):   
        tower_options = sensor_tower_neighbors[current_sensor]
        random.shuffle(tower_options)
        to_tower = tower_options[0][0]
        # figure out the time for the tower ping as t +1 from the last ping
        ping_to_tower = 0
        for key, val in sensor_edges[STARTING_SENSOR].items():
            if ping_to_tower < key:
                ping_to_tower = key
        ping_to_tower = ping_to_tower + 1 
        sensor_edges[STARTING_SENSOR][ping_to_tower] = to_tower
        if current_sensor in sensor_time_check.keys():
            sensor_time_check[current_sensor].append(ping_to_tower)
        else:
            sensor_time_check[current_sensor] = [ping_to_tower]

starting_sensor:  121001920
current_sensor:  121001920
sensor_time_check:  {'121001920': [0]}
sensor_edges:  {'121001920': {0: '104302115'}}
Press Enter to continue...
starting_sensor:  121001920
current_sensor:  104302115
sensor_time_check:  {'121001920': [0], '104302115': [1]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815'}}
Press Enter to continue...
starting_sensor:  121001920
current_sensor:  10331815
sensor_time_check:  {'121001920': [0], '104302115': [1], '10331815': [2]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132'}}
Press Enter to continue...
starting_sensor:  121001920
current_sensor:  122974132
sensor_time_check:  {'121001920': [0], '104302115': [1], '10331815': [2], '122974132': [3]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132', 3: '12059326'}}
Press Enter to continue...
starting_sensor:  128216313
current_sensor:  128216313
sensor_time_check:  {'121001920': [0], '104302115': [1], '10331815': [2], '1

Press Enter to continue...
starting_sensor:  122974132
current_sensor:  122974121
sensor_time_check:  {'121001920': [0, 2], '104302115': [1], '10331815': [2], '122974132': [3, 0], '12059326': [4], '128216313': [0], '130336714': [1], '12100197': [2, 0], '12297418': [3], '12469399': [1], '104115939': [0], '10330844': [1], '103318129': [2], '103318117': [3], '104115918': [4], '126951211': [5], '127620712': [6], '122974121': [1]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132', 3: '12059326', 4: 'MercerIsland'}, '128216313': {0: '130336714', 1: '12100197', 2: '12297418', 3: 'SewardPark'}, '12100197': {0: '12469399', 1: '121001920', 2: '122974132'}, '104115939': {0: '10330844', 1: '103318129', 2: '103318117', 3: '104115918', 4: '126951211', 5: '127620712', 6: 'Eastgate'}, '122974132': {0: '122974121', 1: '10331815'}}
Press Enter to continue...
starting_sensor:  130336727
current_sensor:  130336727
sensor_time_check:  {'121001920': [0, 2], '104302115': [1], '10331

Press Enter to continue...
starting_sensor:  130336714
current_sensor:  128216313
sensor_time_check:  {'121001920': [0, 2], '104302115': [1], '10331815': [2], '122974132': [3, 0], '12059326': [4], '128216313': [0, 1], '130336714': [1, 0], '12100197': [2, 0], '12297418': [3], '12469399': [1], '104115939': [0], '10330844': [1], '103318129': [2], '103318117': [3], '104115918': [4], '126951211': [5], '127620712': [6], '122974121': [1], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132', 3: '12059326', 4: 'MercerIsland'}, '128216313': {0: '130336714', 1: '12100197', 2: '12297418', 3: 'SewardPark'}, '12100197': {0: '12469399', 1: '121001920', 2: '122974132'}, '104115939': {0: '10330844', 1: '103318129', 2: '103318117', 3: '104115918', 4: '126951211', 5: '127620712', 6: 'Eastgate'}, '122974132': {0: '122974121', 1: '10331815'}, '130336727':

Press Enter to continue...
starting_sensor:  103318117
current_sensor:  10330844
sensor_time_check:  {'121001920': [0, 2, 1], '104302115': [1], '10331815': [2], '122974132': [3, 0], '12059326': [4, 0], '128216313': [0, 1], '130336714': [1, 0], '12100197': [2, 0], '12297418': [3], '12469399': [1], '104115939': [0, 1], '10330844': [1, 0, 2], '103318129': [2], '103318117': [3, 0], '104115918': [4], '126951211': [5, 1], '127620712': [6, 2], '122974121': [1], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132', 3: '12059326', 4: 'MercerIsland'}, '128216313': {0: '130336714', 1: '12100197', 2: '12297418', 3: 'SewardPark'}, '12100197': {0: '12469399', 1: '121001920', 2: '122974132'}, '104115939': {0: '10330844', 1: '103318129', 2: '103318117', 3: '104115918', 4: '126951211', 5: '127620712', 6: 'Eastgate'}, '122974132': {0: '122974121', 1: '1

Press Enter to continue...
starting_sensor:  126947610
current_sensor:  126947610
sensor_time_check:  {'121001920': [0, 2, 1], '104302115': [1], '10331815': [2], '122974132': [3, 0], '12059326': [4, 0], '128216313': [0, 1, 2], '130336714': [1, 0], '12100197': [2, 0, 1], '12297418': [3, 3], '12469399': [1, 0], '104115939': [0, 1], '10330844': [1, 0, 2], '103318129': [2, 3], '103318117': [3, 0], '104115918': [4], '126951211': [5, 1], '127620712': [6, 2, 0], '122974121': [1], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7], '126947610': [0]}
sensor_edges:  {'121001920': {0: '104302115', 1: '10331815', 2: '122974132', 3: '12059326', 4: 'MercerIsland'}, '128216313': {0: '130336714', 1: '12100197', 2: '12297418', 3: 'SewardPark'}, '12100197': {0: '12469399', 1: '121001920', 2: '122974132'}, '104115939': {0: '10330844', 1: '103318129', 2: '103318117', 3: '104115918', 4: '126951211', 5: '127620712', 6: 'Eastgate'}

...

Press Enter to continue...
starting_sensor:  1297202
current_sensor:  1302826
sensor_time_check:  {'121001920': [0, 2, 1, 6, 4], '104302115': [1, 3, 2, 5, 0], '10331815': [2, 4, 5, 3, 1, 0], '122974132': [3, 0, 1, 2], '12059326': [4, 0, 2, 3, 1], '128216313': [0, 1, 2], '130336714': [1, 0], '12100197': [2, 0, 1], '12297418': [3, 3, 0], '12469399': [1, 0], '104115939': [0, 1, 4], '10330844': [1, 0, 2, 2, 5], '103318129': [2, 3, 1, 0], '103318117': [3, 0], '104115918': [4, 1, 0], '126951211': [5, 1, 0, 2], '127620712': [6, 2, 0, 3], '122974121': [1, 5, 0, 4], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7], '126947610': [0], '10430211': [1, 0], '1303367': [0], '10314512': [0], '10314533': [1, 0], '1033189': [0], '1035503': [1], '1296535': [2], '1253632': [3], '1240117': [4], '1237377': [5], '1241258': [0], '1256899': [1], '1303963': [2, 1], '1265534': [3], '1236500': [4], '1057319': [0], '1297081': [2], '130

Press Enter to continue...
starting_sensor:  1256899
current_sensor:  1057319
sensor_time_check:  {'121001920': [0, 2, 1, 6, 4], '104302115': [1, 3, 2, 5, 0], '10331815': [2, 4, 5, 3, 1, 0], '122974132': [3, 0, 1, 2], '12059326': [4, 0, 2, 3, 1], '128216313': [0, 1, 2], '130336714': [1, 0], '12100197': [2, 0, 1], '12297418': [3, 3, 0], '12469399': [1, 0], '104115939': [0, 1, 4], '10330844': [1, 0, 2, 2, 5], '103318129': [2, 3, 1, 0], '103318117': [3, 0], '104115918': [4, 1, 0], '126951211': [5, 1, 0, 2], '127620712': [6, 2, 0, 3], '122974121': [1, 5, 0, 4], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7], '126947610': [0], '10430211': [1, 0], '1303367': [0], '10314512': [0], '10314533': [1, 0], '1033189': [0], '1035503': [1], '1296535': [2], '1253632': [3], '1240117': [4], '1237377': [5], '1241258': [0], '1256899': [1, 0], '1303963': [2, 1, 0], '1265534': [3], '1236500': [4], '1057319': [0, 1], '1297081': 

Press Enter to continue...
starting_sensor:  1256899
current_sensor:  1030718
sensor_time_check:  {'121001920': [0, 2, 1, 6, 4], '104302115': [1, 3, 2, 5, 0], '10331815': [2, 4, 5, 3, 1, 0], '122974132': [3, 0, 1, 2], '12059326': [4, 0, 2, 3, 1], '128216313': [0, 1, 2], '130336714': [1, 0], '12100197': [2, 0, 1], '12297418': [3, 3, 0], '12469399': [1, 0], '104115939': [0, 1, 4], '10330844': [1, 0, 2, 2, 5], '103318129': [2, 3, 1, 0], '103318117': [3, 0], '104115918': [4, 1, 0], '126951211': [5, 1, 0, 2], '127620712': [6, 2, 0, 3], '122974121': [1, 5, 0, 4], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7], '126947610': [0], '10430211': [1, 0], '1303367': [0], '10314512': [0], '10314533': [1, 0], '1033189': [0], '1035503': [1], '1296535': [2], '1253632': [3], '1240117': [4], '1237377': [5], '1241258': [0], '1256899': [1, 0], '1303963': [2, 1, 0], '1265534': [3], '1236500': [4], '1057319': [0, 1], '1297081': 

Press Enter to continue...
starting_sensor:  1032128
current_sensor:  1309765
sensor_time_check:  {'121001920': [0, 2, 1, 6, 4], '104302115': [1, 3, 2, 5, 0], '10331815': [2, 4, 5, 3, 1, 0], '122974132': [3, 0, 1, 2], '12059326': [4, 0, 2, 3, 1], '128216313': [0, 1, 2], '130336714': [1, 0], '12100197': [2, 0, 1], '12297418': [3, 3, 0], '12469399': [1, 0], '104115939': [0, 1, 4], '10330844': [1, 0, 2, 2, 5], '103318129': [2, 3, 1, 0], '103318117': [3, 0], '104115918': [4, 1, 0], '126951211': [5, 1, 0, 2], '127620712': [6, 2, 0, 3], '122974121': [1, 5, 0, 4], '130336727': [0], '1299972': [1], '1306673': [2], '1239093': [3], '1265886': [4], '1270795': [5], '1239192': [6], '1270717': [7], '126947610': [0], '10430211': [1, 0], '1303367': [0], '10314512': [0], '10314533': [1, 0], '1033189': [0], '1035503': [1], '1296535': [2], '1253632': [3], '1240117': [4], '1237377': [5], '1241258': [0], '1256899': [1, 0], '1303963': [2, 1, 0], '1265534': [3], '1236500': [4], '1057319': [0, 1], '1297081': 

KeyboardInterrupt: 

In [45]:
sensor_edges

{'121001920': {0: '104302115',
  1: '10331815',
  2: '122974132',
  3: '12059326',
  4: 'MercerIsland'},
 '128216313': {0: '130336714', 1: '12100197', 2: '12297418', 3: 'SewardPark'},
 '12100197': {0: '12469399', 1: '121001920', 2: '122974132'},
 '104115939': {0: '10330844',
  1: '103318129',
  2: '103318117',
  3: '104115918',
  4: '126951211',
  5: '127620712',
  6: 'Eastgate'},
 '122974132': {0: '122974121', 1: '10331815'},
 '130336727': {0: '1299972',
  1: '1306673',
  2: '1239093',
  3: '1265886',
  4: '1270795',
  5: '1239192',
  6: '1270717',
  7: 'PioneerSquare'},
 '130336714': {0: '128216313', 1: '121001920'},
 '12059326': {0: '121001920', 1: '10331815'},
 '10330844': {0: '126951211', 1: '127620712', 2: 'MSFTWest'},
 '103318117': {0: '104115939', 1: '10330844', 2: '103318129', 3: '104115918'},
 '127620712': {0: '10330844'},
 '12469399': {0: '12100197', 1: '128216313', 2: '12297418', 3: 'Eastgate'},
 '126947610': {0: '10430211',
  1: '12059326',
  2: '104302115',
  3: '10331815

In [56]:
sensor_edges["12100197"]

{0: '12469399', 1: '121001920', 2: '122974132'}

In [54]:
sensor_time_check["122974132"]

[0, 2, 1, 6, 4]

In [57]:
sensor_edges["122974132"]

{0: '122974121', 1: '10331815'}

In [58]:
sensor_time_check["10331815"]

[2, 4, 5, 3, 1, 0]

In [59]:
sensor_edges["10331815"]

{0: '122974132'}

In [52]:
sensor_edges["122974132"]

{0: '122974121', 1: '10331815'}

In [150]:
def input_needed(location, t, message, STARTING_SENSOR, edge_list, sensor_path):
    """ helper function for inspection"""
    print("SOMETHING ISN'T RIGHT, checking in", location)
    print("iteration: ", t)
    print(message)
    print("Starting Sensor: ", STARTING_SENSOR)
    print("Edge List: ")
    pprint(edge_list)
    print("Sensor Path: ")
    pprint(sensor_path)
    input("what do you want to do?")
    

In [163]:
def edge_exists(current_sensor, edge_list, t):
    """
    check if there already is an edge at time t leaving this sensor
    Structure:
        {vertex: {0: next0}, 
         next0: {1: next1},
         next1: {2: next2}
    """
    if(current_sensor in edge_list.keys()):
        if(t in edge_list[current_sensor].keys()):
            return True
    return False

def get_adjacent_vertex(current_sensor, edge_list, t):
    """
    get the vertex connect to the current sensor at time t
    """
    if(current_sensor in edge_list.keys()):
        if(t in edge_list[current_sensor].keys()):
            return edge_list[current_sensor][t]
    return None

def is_tower(current_sensor, tower_names):
    """
    return true of the sensor is a tower
    """
    if(current_sensor in tower_names):
        return True
    else:
        return False

    
def update_edge_list(current_sensor, edge_list, t, location, message, STARTING_SENSOR, sensor_path, to_edge):
    """
    update in memory graph data structure
    Structure:
        {vertex: {0: next0}, 
         next0: {1: next1},
         next1: {2: next2}
    """
    # current_sensor --- t ---> to_edge
    if current_sensor in edge_list.keys():
        if t in edge_list[current_sensor].keys():
            #input_needed(location, t, message, STARTING_SENSOR, edge_list, sensor_path)
            print("Variable Dump")
            print("Start: ", STARTING_SENSOR)
            print("Current: ", current_sensor)
            print("to_Edge: ", to_edge)
            print("Time: ", t)
            print("******** SP ********")
            pprint(sensor_path)
            print("******** EL ********")
            pprint(edge_list)
            print(location)
            print(message)
            
            raise("didn't expect an edge here")
        else:
            edge_list[current_sensor][t] = to_edge
    else:
        edge_list[current_sensor] = {t: to_edge}
    return edge_list

In [189]:
# {vertex : {0:next0, 1:next1, 2:next2}}
sensor_path = {}
# {vertex: {0: next0}, 
#  next0: {1: next1},
#  next1: {2: next2}
edge_list = {}

for i, (STARTING_SENSOR, val) in enumerate(SENSORS.items()):
    visited_on_path = [STARTING_SENSOR]  # avoid self loops
    current_sensor = STARTING_SENSOR # setting our starting place
    sensor_path[STARTING_SENSOR] = {} # tracing the path from the starting sensor to a tower
    next_vertex = None
    time = 0
    for t in range(0, 5): # only going 5
        time = t
        # need to know if there is a tower path
        edge_check = edge_exists(current_sensor, edge_list, time)
        if(edge_check):
            # exit the attempt to find another edge based on time if there already is one
            break;
        cycle_detector = 0
        while(cycle_detector < 5): ## 5 because we have 5 close neighbors in each sensor neighborhood
            # pick a neighbor randomly from current sensor's sensor_neighbors
            random.shuffle(sensor_neighbors[current_sensor])
            to_edge = sensor_neighbors[current_sensor][0][0]
            if to_edge in visited_on_path:
                # want to try again at the while loop
                cycle_detector += 1
            else: 
                # update data structures at time t
                visited_on_path.append(to_edge)
                sensor_path[STARTING_SENSOR][t] = to_edge
                edge_list = update_edge_list(current_sensor, edge_list, t, 
                                             "Location: inside cycle detector while loop", 
                                             "CONSTRUCTION ERROR: tried to add edge to edge list but one already existed", 
                                             STARTING_SENSOR, sensor_path,
                                             to_edge) 
                current_sensor = to_edge
                break
        # if cycle_detector < 5, then we sucessfully found an edge
        # This means that we just want to continue with a new t value
        if(cycle_detector >= 5):
            # you didn't get through all of t
            # you just want to complete this path
            break
    edge_check = edge_exists(current_sensor, edge_list, time)
    if(edge_check):
        # this means that we already have an edge @ time = time
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
        if is_tower(next_vertex, tower_names):
            # then we need to end the current path at the tower
            sensor_path[STARTING_SENSOR][time+1] = next_vertex
        else:
            # not a tower; walk until you get a tower or None
            while(next_vertex):
                current_sensor = next_vertex
                time = time + 1
                next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
            # while loop broke when next_vertex was None
            # check if current_sensor is a tower
            if(is_tower(current_sensor, tower_names)):
                # we have a valid path to a tower
                pass
            else:
                # make a tower path
                # current_sensor is a sensor
                # there is no endpoint at time tie
                # make it a tower
                # add it to the data structures
                random.shuffle(sensor_tower_neighbors[current_sensor])
                to_tower = sensor_tower_neighbors[current_sensor][0][0]
                edge_list = update_edge_list(current_sensor, edge_list, time, 
                                         "location: there was an edge after the for, and walked the graph but didn't find a tower", 
                                         "CONSTRUCTION ERROR: wasn't expecting an edge at this time for a tower ", 
                                         STARTING_SENSOR, sensor_path,
                                         to_tower)
    else:
        # there is not an edge at time
        # to connect it to a tower and move on
        random.shuffle(sensor_tower_neighbors[current_sensor])
        to_tower = sensor_tower_neighbors[current_sensor][0][0]
        sensor_path[STARTING_SENSOR][time] = to_tower
        edge_list = update_edge_list(current_sensor, edge_list, time, 
                                     "location: last update statement for adding a tower", 
                                     "CONSTRUCTION ERROR: wasn't expecting an edge at this time for a tower ", 
                                     STARTING_SENSOR, sensor_path,
                                     to_tower) 
    # VALIDATE CONSTRUCTION FOR THE STARTING VERTEX
    time = 0
    current_sensor = STARTING_SENSOR
    path = [current_sensor]
    next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    while(next_vertex):
        path.append(next_vertex)
        current_sensor = next_vertex
        time = time + 1
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    if(is_tower(current_sensor, tower_names)):
        # we created a proper tree
        pass
    else:
        print("INCORRECT TREE FOUND: variable dump")
        print("Variable Dump")
        print("Start: ", STARTING_SENSOR)
        print("Current: ", current_sensor)
        print("Next: ", next_vertex)
        print("path: ", path)
        print("Time: ", time)
        print("******** SP ********")
        pprint(sensor_path)
        print("******** EL ********")
        pprint(edge_list)
        input("press enter")

INCORRECT TREE FOUND: variable dump
Variable Dump
Start:  128216313
Current:  121001920
Next:  None
path:  ['128216313', '130336714', '12100197', '12297418', '12469399', '121001920']
Time:  5
******** SP ********
{'121001920': {0: '10331815',
               1: '104302115',
               2: '12059326',
               3: '122974132',
               4: 'Eastgate'},
 '128216313': {0: '130336714',
               1: '12100197',
               2: '12297418',
               3: '12469399',
               4: 'Newcastle'}}
******** EL ********
{'10331815': {1: '104302115'},
 '104302115': {2: '12059326'},
 '12059326': {3: '122974132'},
 '121001920': {0: '10331815', 4: 'Newcastle'},
 '12100197': {2: '12297418'},
 '122974132': {4: 'Eastgate'},
 '12297418': {3: '12469399'},
 '12469399': {4: '121001920'},
 '128216313': {0: '130336714'},
 '130336714': {1: '12100197'}}


KeyboardInterrupt: 

In [ ]:
tests = []
for towerStart, path in sensor_path.items():
    last = towerStart
    for time, tower in path.items():
        last = tower
    if(is_tower(last, tower_names)):
        pass
    else:
        tests.append(towerStart)


In [179]:
edge_list["121001920"]

{0: '12059326', 4: '104302115', 1: '12059326', 2: '12059326'}

In [181]:
bad_cases = {}
for test in tests:
    time = 0
    current_sensor = test
    path = [current_sensor]
    next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    while(next_vertex):
        path.append(next_vertex)
        current_sensor = next_vertex
        time = time + 1
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    if(is_tower(current_sensor, tower_names)):
        pass
    else:
        pprint(path)
        bad_cases[test] = path
        input("press enter")

['12297418', '128216313', '12100197', '130336714', '12469399', '121001920']
press enter
['1012192', '1049589', '1034751', '1034753', '1056955', '1031437']
press enter
['1297264', '1302824', '1280723', '1246797', '1303687', '1011408']
press enter
['1033098', '1033097', '1248249', '1248210', '1280634', '1302928']
press enter
['1027842', '1253132', '1061733', '1224666', '1064041', '1236540']
press enter


In [182]:
sensor_path["12297418"]

{0: '128216313', 1: '12100197', 2: '130336714', 3: '12469399', 4: '121001920'}

In [183]:
edge_list["121001920"]

{0: '12059326', 4: '104302115', 1: '12059326', 2: '12059326'}

In [ ]:
# {vertex : {0:next0, 1:next1, 2:next2}}
sensor_path = {}
# {vertex: {0: next0}, 
#  next0: {1: next1},
#  next1: {2: next2}
edge_list = {}

for i, (STARTING_SENSOR, val) in enumerate(SENSORS.items()):
    visited_on_path = [STARTING_SENSOR]  # avoid self loops
    current_sensor = STARTING_SENSOR # setting our starting place
    sensor_path[STARTING_SENSOR] = {} # tracing the path from the starting sensor to a tower
    next_vertex = None
    time = 0
    for t in range(0, 5): # only going 5
        time = t
        # need to know if there is a tower path
        edge_check = edge_exists(current_sensor, edge_list, time)
        if(edge_check):
            # exit the attempt to find another edge based on time if there already is one
            break;
        cycle_detector = 0
        while(cycle_detector < 5): ## 5 because we have 5 close neighbors in each sensor neighborhood
            # pick a neighbor randomly from current sensor's sensor_neighbors
            random.shuffle(sensor_neighbors[current_sensor])
            to_edge = sensor_neighbors[current_sensor][0][0]
            if to_edge in visited_on_path:
                # want to try again at the while loop
                cycle_detector += 1
            else: 
                # update data structures at time t
                visited_on_path.append(to_edge)
                sensor_path[STARTING_SENSOR][t] = to_edge
                edge_list = update_edge_list(current_sensor, edge_list, t, 
                                             "Location: inside cycle detector while loop", 
                                             "CONSTRUCTION ERROR: tried to add edge to edge list but one already existed", 
                                             STARTING_SENSOR, sensor_path,
                                             to_edge) 
                current_sensor = to_edge
                break
        # if cycle_detector < 5, then we sucessfully found an edge
        # This means that we just want to continue with a new t value
        if(cycle_detector >= 5):
            # you didn't get through all of t
            # you just want to complete this path
            break
    # What guarantees do we have after the for loop?
    # 1. for the starting sensor, we have 
    edge_check = edge_exists(current_sensor, edge_list, time)
    if(edge_check):
        # this means that we already have an edge @ time = time
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
        if is_tower(next_vertex, tower_names):
            # then we need to end the current path at the tower
            sensor_path[STARTING_SENSOR][time+1] = next_vertex
            edge_list = update_edge_list(current_sensor, edge_list, time+1, 
                                             "Location: inside cycle detector while loop", 
                                             "CONSTRUCTION ERROR: tried to add edge to edge list but one already existed", 
                                             STARTING_SENSOR, sensor_path,
                                             next_vertex) 
        else:
            # not a tower; walk until you get a tower or None
            # we have walked the edge at time time
            # we have a valid vertex at next_vertex
            while(next_vertex):
                current_sensor = next_vertex
                time = time + 1
                next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
            # while loop broke when next_vertex was None
            # check if current_sensor is a tower
            if(is_tower(current_sensor, tower_names)):
                # we have a valid path to a tower
                pass
            else:
                # make a tower path
                # current_sensor is a sensor
                # there is no endpoint at time tie
                # make it a tower
                # add it to the data structures
                random.shuffle(sensor_tower_neighbors[current_sensor])
                to_tower = sensor_tower_neighbors[current_sensor][0][0]
                sensor_path[STARTING_SENSOR][time] = to_tower
                edge_list = update_edge_list(current_sensor, edge_list, time, 
                                         "location: there was an edge after the for, and walked the graph but didn't find a tower", 
                                         "CONSTRUCTION ERROR: wasn't expecting an edge at this time for a tower ", 
                                         STARTING_SENSOR, sensor_path,
                                         to_tower)
    else:
        # there is not an edge at time
        # to connect it to a tower and move on
        random.shuffle(sensor_tower_neighbors[current_sensor])
        to_tower = sensor_tower_neighbors[current_sensor][0][0]
        sensor_path[STARTING_SENSOR][time+1] = to_tower
        edge_list = update_edge_list(current_sensor, edge_list, time+1, 
                                     "location: last update statement for adding a tower", 
                                     "CONSTRUCTION ERROR: wasn't expecting an edge at this time for a tower ", 
                                     STARTING_SENSOR, sensor_path,
                                     to_tower) 
    # VALIDATE CONSTRUCTION FOR THE STARTING VERTEX
    time = 0
    current_sensor = STARTING_SENSOR
    path = [current_sensor]
    next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    while(next_vertex):
        path.append(next_vertex)
        current_sensor = next_vertex
        time = time + 1
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    if(is_tower(current_sensor, tower_names)):
        # we created a proper tree
        pass
    else:
        print("INCORRECT TREE FOUND: variable dump")
        print("Variable Dump")
        print("Start: ", STARTING_SENSOR)
        print("Current: ", current_sensor)
        print("Next: ", next_vertex)
        print("path: ", path)
        print("Time: ", time)
        print("******** SP ********")
        pprint(sensor_path)
        print("******** EL ********")
        pprint(edge_list)
        input("press enter")

INCORRECT TREE FOUND: variable dump
Variable Dump
Start:  128216313
Current:  10331815
Next:  None
path:  ['128216313', '12297418', '12100197', '121001920', '104302115', '10331815']
Time:  5
******** SP ********
{'121001920': {0: '104302115',
               1: '122974132',
               2: '12059326',
               3: '10331815',
               4: '122974121',
               5: 'Bellevue'},
 '128216313': {0: '12297418',
               1: '12100197',
               2: '121001920',
               3: '104302115',
               4: '10331815'}}
******** EL ********
{'10331815': {4: '122974121'},
 '104302115': {1: '122974132', 4: '10331815'},
 '12059326': {3: '10331815'},
 '121001920': {0: '104302115', 3: '104302115'},
 '12100197': {2: '121001920'},
 '122974121': {5: 'Bellevue'},
 '122974132': {2: '12059326'},
 '12297418': {1: '12100197'},
 '128216313': {0: '12297418'}}
